In [2]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
transforms = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize(mean = [0.485, 0.456, 0.456], std=[0.229, 0.224, 0.225])])
train = datasets.ImageFolder('./cats_and_dogs_filtered/train', transform= transforms)
test = datasets.ImageFolder('./cats_and_dogs_filtered/validation', transform= transforms)
train_loader = DataLoader(train, batch_size= 128, shuffle= True)
test_loader = DataLoader(test, batch_size= 128)

In [4]:
class dropout(nn.Module):
    def __init__(self, p = 0.5):
        super().__init__()
        self.p = p
    def forward(self, x):
        if self.training:
            mask = (torch.rand_like(x) > self.p).float()
            return (x * mask) / (1 - self.p)
        return x
    
class CNN(nn.Module):
    def __init__(self, p = 0.5):
        super().__init__()
        self.net = nn.Sequential(nn.Conv2d(3, 32, kernel_size=3, padding=1),
                                 nn.Conv2d(32, 64, kernel_size=3, padding=1),
                                 nn.Conv2d(64, 128, kernel_size=3, padding=1),
                                 nn.MaxPool2d(kernel_size=2, stride=2),
                                 nn.MaxPool2d(kernel_size=2, stride=2),
                                 nn.MaxPool2d(kernel_size=2, stride=2))
        self.classify_head = nn.Sequential(nn.Flatten(),
                                           nn.Linear(128 * 28 * 28, 512),
                                           dropout(p),
                                           nn.Linear(512, 2))
    def forward(self, x):
        return self.classify_head(self.net(x))

model = CNN()
model.to('cuda')


CNN(
  (net): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classify_head): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=100352, out_features=512, bias=True)
    (2): dropout()
    (3): Linear(in_features=512, out_features=2, bias=True)
  )
)

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= 0.001)

In [6]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
print(f'Epoch - {epoch}, loss = {running_loss}')

Epoch - 9, loss = 10.083131730556488


In [7]:
all_preds, all_labels = [], []
model.eval()
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = model(input)
        val, ind = torch.max(output, dim = 1)
        all_preds.extend(ind.to('cpu'))
        all_labels.extend(target.to('cpu'))
from sklearn.metrics import accuracy_score
print(accuracy_score(all_preds, all_labels))


0.622
